### Spin Test 
This script projects data to surface space and performs a spin test

#### Import Packages

In [63]:
import numpy as np
import pandas as pd
import netneurotools.stats as stats
from neuromaps.datasets import fetch_atlas
from nilearn.datasets import fetch_atlas_schaefer_2018
from netneurotools.datasets import fetch_schaefer2018
import neuromaps
import nibabel as nib
from neuromaps.parcellate import Parcellater
from neuromaps import transforms
from neuromaps import nulls
from nilearn.plotting import plot_surf
from nilearn.surface import load_surf_data
from nilearn import surface, plotting, datasets
from neuromaps import datasets, images, nulls, resampling, stats

#### Set Paths and Variables

In [75]:
dataset = 'longglucest_outputmeasures2'
atlas = 'Schaefer2018_1000Parcels_17Networks'
nmaps = ["NMDA", "mGluR5", "GABA"]
maps = ["cest", "NMDA", "mGluR5", "GABA"]
n_permute = 10

# Fetch Schaefer 2018 atlas coordinates (for example)
schaefer = fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17)
labels = schaefer['labels']  # These are the MNI coordinates of the parcels
labels = [label.decode('utf-8') for label in labels]
schaefernifti = schaefer['maps']
abagen = datasets.fetch_annotation(source='abagen', desc='genepc1',
                                       space='fsaverage', den='10k')


In [64]:
dataset = 'longglucest_outputmeasures2'
atlas = 'Schaefer2018_1000Parcels_17Networks'
nmaps = ["NMDA", "mGluR5", "GABA"]
maps = ["cest", "NMDA", "mGluR5", "GABA"]
n_permute = 10

# Set paths
inpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/nmaps/analyses/" + atlas
outpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/nmaps/analyses/" + atlas

# Read in data
receptor_df = pd.read_csv("/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/results/receptor_data_scale1000_17.csv", sep=',')
means= pd.read_csv(inpath + '/means_subjectnmaps_' + dataset + '_' + atlas + '.csv', sep=',')
means["Parcel"] = means["Parcel"].str.replace(' NZMean', '', regex=False)
means.set_index('Parcel', inplace = True)

# Fetch Schaefer 2018 atlas coordinates (for example)
schaefer = fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17)
labels = schaefer['labels']  # These are the MNI coordinates of the parcels
labels = [label.decode('utf-8') for label in labels]
schaefernifti = schaefer['maps']
abagen = datasets.available_annotations(source='abagen', desc='genepc1',
                                       space='fsaverage', den='10k')
print(abagen)
#abagenmap = abagen['maps']

print(means)
print(schaefernifti)

OSError: [Errno 6] Device not configured: '/Users/pecsok/Desktop/ImageData/PMACS_remote/data/nmaps/analyses/Schaefer2018_1000Parcels_17Networks/means_subjectnmaps_longglucest_outputmeasures2_Schaefer2018_1000Parcels_17Networks.csv'

### Generate permutations

In [77]:
# Project to surface
schaefer_surf1 = transforms.mni152_to_fsaverage(schaefer['maps'], fsavg_density='10k', method='nearest')
# Permute parcels
permutations1 = nulls.vazquez_rodriguez(data=None, atlas='fsaverage', density='10k', 
                                  n_perm=n_permute, seed=1234, parcellation=schaefer_surf1)
# Inspect dataframe
permutations_df = pd.DataFrame(permutations1)
permutations_df.to_csv('~/Desktop/permutations400.csv', index=True)
print(permutations_df)


# Project to surface
schaefer_surf2, abagen = resampling.resample_images(schaefernifti, abagen, 'MNI152', 'fsaverage')
permutations2 = nulls.vazquez_rodriguez(data=None, atlas='fsaverage', density='10k', 
                                  n_perm=n_permute, seed=1234, parcellation=schaefer_surf2)

permutations_df2 = pd.DataFrame(permutations2)
permutations_df2.to_csv('~/Desktop/permutations.csv', index=True)
print(permutations_df2)



#permutationsvol = nulls.vazquez_rodriguez(data=None, atlas='mni152', density='1mm',
#                                 n_perm=n_permute, seed=1234, parcellation=schaefernifti)


fsavg_atlas = fetch_atlas('fsaverage', '10k')
fsavg_gii = nib.load(fsavg_atlas['sphere'].L)
vertices = fsavg_gii.agg_data('NIFTI_INTENT_POINTSET')
print(vertices.shape[0])

"""

fsavg_l, fsavg_r = fsavg_gii[0], fsavg_gii[1]

fslr_left_infl = nib.load(fslr['inflated'].L)
vertices = fslr_left_infl.agg_data('NIFTI_INTENT_POINTSET')



print(fsavg_gii[0])


#print(lpermutations_df.columns))
print(len(permutations_df.index))

#Fix not sure why there are 1001 rows in permutation matrix. 
#  Why is there an extra parcel??
"""

       0    1    2    3    4    5    6    7    8    9
0     29  171  121   35   64   68   45  152    2  149
1    131  192   19   24   56  174  125   35  120    1
2    129  116   15   23   73  137  136   34  167   62
3    101  192   17   11   57   87  126   77   90   61
4    129  116   14   11   74  123  150   28  195   60
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
398  348  216  222  209  345  253  234  330  385  215
399  384  219  365  217  344  250  241  343  350  215
400  291  389  369  221  310  247  245  327  349  218
401  384  389  365  217  363  250  283  343  307  216
402  325  389  367  218  309  248  238  362  351  216

[403 rows x 10 columns]
         0     1     2     3     4     5     6     7     8     9
0      312  2168  1626   423  1013   854   568  1995    17  1899
1     1550  1418  1397  2175   435  2271   219   578   961  1338
2     1117    52  1595   458  2398    82   974  1188     0   834
3     2277  2306  1699  1126  1996  2234  1310  2401   719  1511
4 

"\n\nfsavg_l, fsavg_r = fsavg_gii[0], fsavg_gii[1]\n\nfslr_left_infl = nib.load(fslr['inflated'].L)\nvertices = fslr_left_infl.agg_data('NIFTI_INTENT_POINTSET')\n\n\n\nprint(fsavg_gii[0])\n\n\n#print(lpermutations_df.columns))\nprint(len(permutations_df.index))\n\n#Fix not sure why there are 1001 rows in permutation matrix. \n#  Why is there an extra parcel??\n"

In [56]:
receptor_df = receptor_df.reset_index(drop=True)
receptor_df.index = pd.RangeIndex(start=1, stop=len(receptor_df) + 1, step=1)

         NMDA    mGluR5     GABAa        D2
0    0.222071 -1.676590  0.454865 -1.793413
1   -0.498181 -1.582004  0.778041 -1.725353
2    0.901576 -0.612121  2.141349 -1.079382
3   -1.250976 -2.019832 -0.681790 -1.734788
4    0.924391 -0.341650  0.853886  0.268522
..        ...       ...       ...       ...
995  0.906115  0.748027  0.435516  0.148707
996 -0.093347  0.810461 -0.015695  0.217057
997  0.859066  0.746511  0.069180  0.635338
998  0.825170  1.057345  0.635444  0.371140
999  0.219509  1.022109  0.470702  0.717365

[1000 rows x 4 columns]
          NMDA    mGluR5     GABAa        D2
1     0.222071 -1.676590  0.454865 -1.793413
2    -0.498181 -1.582004  0.778041 -1.725353
3     0.901576 -0.612121  2.141349 -1.079382
4    -1.250976 -2.019832 -0.681790 -1.734788
5     0.924391 -0.341650  0.853886  0.268522
...        ...       ...       ...       ...
996   0.906115  0.748027  0.435516  0.148707
997  -0.093347  0.810461 -0.015695  0.217057
998   0.859066  0.746511  0.069180  0.6353

In [60]:
# Function to scramble the dataframe rows based on randomization matrix
def scramble_rows(df, rand_matrix):
    scrambled_dfs = []
    
    # Loop through each column of the randomization matrix (each column is a new randomization)
    for col_idx in range(rand_matrix.shape[1]):
        # Check that the indices are valid
        if rand_matrix[:, col_idx].max() >= len(df):
            raise ValueError(f"Randomization index exceeds available rows in df: {rand_matrix[:, col_idx].max()}")
        
        scrambled_df = df.iloc[rand_matrix[:, col_idx]].reset_index(drop=True)  # Use row indices from the column
        scrambled_dfs.append(scrambled_df)
        
    return scrambled_dfs

# Assume permutations_df is a DataFrame and remove the last row
permutations_df = permutations_df.iloc[:-1]
print(permutations_df.shape)

# Apply the function to scramble your dataframe
scrambled_dataframes = scramble_rows(receptor_df, permutations_df.values)

# Example: Access the first scrambled dataframe
print(scrambled_dataframes[0])



(1000, 10)


ValueError: Randomization index exceeds available rows in df: 1000

In [ ]:
# Now, input means df. 
# Rename parcels by parcel number and put rows in proper order.




receptor_df.index = labels
receptor_df.index.name = 'Parcel'

# Chop up receptor_df by map
NMDAmat = receptor_df[["NMDA"]]
GABAmat = receptor_df[["GABAa"]]
mGluR5mat = receptor_df[["mGluR5"]]

In [ ]:
# CEST DATA
datapath = '/Users/pecsok/projects/Neuromaps/hansen_receptors/'
figpath = '/Users/pecsok/projects/GluCEST-fMRI/glucest-rsfmri/fmri_pipeline/parcellated_pipeline/figures'
cestavg_df = pd.DataFrame(index=range(1, 1001), columns=['CESTavg'])
cestavg_df['CESTavg'] = np.nan  # Initialize all values to 0
for _, row in cestdf.iterrows():
    # Extract parcel ID number using regex
    match = re.search(r'(\d+)', row['parcel'])
    if match:
        parcel_id = int(match.group(1))
        # Add CESTavg data to corresponding row in the new DataFrame
        if 1 <= parcel_id <= 1000:  # Ensure the parcel ID is within range
            cestavg_df.at[parcel_id, 'CESTavg'] = row['CESTavg']
            
for i, row in cestavg_df.iterrows():
    parcel_id = i
    value = row['CESTavg']
print(cestavg_df)

